Importar bibliotecas necesarias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tensorflow.keras.models import load_model


In [2]:
!pip install scikeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [3]:
!pip install scikeras[tensorflow]


In [4]:
pip install tensorflow


In [5]:
!pip install --upgrade scikit-learn


In [6]:
!pip install --upgrade tensorflow scikeras


Cargar y preprocesar dataset

In [7]:
# Cargar el dataset
data_url = "/content/framingham_clean.csv"
data = pd.read_csv(data_url)

# Reemplazar 'Missing' con NaN para poder manejarlos
data.replace('Missing', np.nan, inplace=True)

# Rellenar valores NaN con la mediana de cada columna
data.fillna(data.median(), inplace=True)


# Separar características y variable objetivo
X = data.drop(columns=['TenYearCHD', 'Unnamed: 0'])
y = data['TenYearCHD']

# Convertir variables categóricas a numéricas
X['gender'] = X['gender'].replace({'M': 0, 'F': 1}).astype(int)
X['currentSmoker'] = X['currentSmoker'].replace({'NO': 0, 'YES': 1}).astype(int)

# Estandarizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


FileNotFoundError: [Errno 2] No such file or directory: '/content/framingham_clean.csv'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_model(optimizer='adam', init_mode='uniform', activation='relu', neurons=8):
    model = Sequential()
    model.add(Dense(neurons, input_dim=17, kernel_initializer=init_mode, activation=activation))
    model.add(Dense(neurons, kernel_initializer=init_mode, activation=activation))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = create_model()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8229 - loss: 0.6823 - val_accuracy: 0.8502 - val_loss: 0.6190
Epoch 2/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8432 - loss: 0.5684 - val_accuracy: 0.8502 - val_loss: 0.4179
Epoch 3/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8422 - loss: 0.4212 - val_accuracy: 0.8502 - val_loss: 0.3825
Epoch 4/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8431 - loss: 0.3934 - val_accuracy: 0.8502 - val_loss: 0.3784
Epoch 5/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8438 - loss: 0.3956 - val_accuracy: 0.8502 - val_loss: 0.3758
Epoch 6/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8462 - loss: 0.3927 - val_accuracy: 0.8502 - val_loss: 0.3739
Epoch 7/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8384 - loss: 0.3968 - val_accuracy: 0.8502 - val_loss: 0.3722
Epoch 8/100
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8529 - loss: 0.3697 - val_accuracy: 0.8502 - v

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Loss: {test_loss}")


Test Accuracy: 0.849056601524353
Test Loss: 0.39554914832115173


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop

def create_model(optimizer, neurons):
    model = Sequential()
    model.add(Dense(neurons, input_dim=17, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

optimizers = [Adam, RMSprop]  # Usamos las clases, no las instancias
neurons = [8, 12]
epochs = [50, 100]
batch_sizes = [16, 32]

best_accuracy = 0
best_params = {}

for optimizer_class in optimizers:
    for neuron in neurons:
        for epoch in epochs:
            for batch_size in batch_sizes:
                optimizer = optimizer_class()  # Crear una nueva instancia del optimizador
                model = create_model(optimizer=optimizer, neurons=neuron)
                history = model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, validation_split=0.2, verbose=0)
                _, accuracy = model.evaluate(X_test, y_test, verbose=0)
                print(f"Accuracy: {accuracy} - Optimizer: {optimizer}, Neurons: {neuron}, Epochs: {epoch}, Batch size: {batch_size}")

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'optimizer': optimizer_class.__name__, 'neurons': neuron, 'epochs': epoch, 'batch_size': batch_size}

print(f"Best Accuracy: {best_accuracy} with params: {best_params}")


Accuracy: 0.8466981053352356 - Optimizer: <keras.src.optimizers.adam.Adam object at 0x7c303ef27cd0>, Neurons: 8, Epochs: 50, Batch size: 16
Accuracy: 0.8466981053352356 - Optimizer: <keras.src.optimizers.adam.Adam object at 0x7c303f09bdf0>, Neurons: 8, Epochs: 50, Batch size: 32
Accuracy: 0.847484290599823 - Optimizer: <keras.src.optimizers.adam.Adam object at 0x7c303f2c1ae0>, Neurons: 8, Epochs: 100, Batch size: 16
Accuracy: 0.849056601524353 - Optimizer: <keras.src.optimizers.adam.Adam object at 0x7c303ec8f5b0>, Neurons: 8, Epochs: 100, Batch size: 32
Accuracy: 0.8514150977134705 - Optimizer: <keras.src.optimizers.adam.Adam object at 0x7c30e1af8b20>, Neurons: 12, Epochs: 50, Batch size: 16
Accuracy: 0.8466981053352356 - Optimizer: <keras.src.optimizers.adam.Adam object at 0x7c303e8676a0>, Neurons: 12, Epochs: 50, Batch size: 32
Accuracy: 0.8451257944107056 - Optimizer: <keras.src.optimizers.adam.Adam object at 0x7c303e8b7d90>, Neurons: 12, Epochs: 100, Batch size: 16
Accuracy: 0.8474